In [43]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('data/test_data.csv')
START_DATE = pd.to_datetime('1990-01-01')
END_DATE = pd.to_datetime('2050-01-01')

STRING_FORMATS = [
    '%Y-%m-%d %H:%M:%S',
    '%d-%m-%Y',
    '%m/%d/%Y %H:%M %p'
]

def datetime_parser(date_value):
    """
    A single, efficient function to parse a value that could be
    a Unix timestamp or one of several string formats.
    """
    numeric_val = pd.to_numeric(date_value, errors='coerce')
    if pd.notna(numeric_val):
        try:
            ts = pd.to_datetime(numeric_val, unit='s')
            if START_DATE <= ts <= END_DATE:
                return ts
        except (ValueError, pd.errors.OutOfBoundsDatetime):
            pass

    if isinstance(date_value, str):
        for fmt in STRING_FORMATS:
            try:
                return pd.to_datetime(date_value, format=fmt)
            except (ValueError, TypeError):
                continue

    return pd.NaT

df['timestamp'] = df['timestamp'].apply(datetime_parser)

df = df.sort_values(['user_id', 'timestamp'])

df = df.dropna(subset=['org_id', 'user_id'])

first_orgs = df.groupby('user_id')['org_id'].transform('first')
df = df[df['org_id'] == first_orgs]

df['timestamp'] = df['timestamp'].fillna(pd.to_datetime('1990-01-01 00:00:00'))

df_timestamps = df['timestamp']

df_credits = df['credits']

df_string_cols = df.select_dtypes('object')
for col in df_string_cols.columns:
    df_string_cols[col] = df_string_cols[col].str.strip()
    df_string_cols[col] = df_string_cols[col].str.lower()
df_string_cols

df = pd.concat([df_string_cols, df_credits, df_timestamps], axis=1)

df['credits'] = df['credits'].fillna(1.0)

df['credit_type'] = df['credit_type'].fillna('default')

df

,org_id,user_id,credit_type,action,credits,timestamp
500,org_001,user_000,report_download,add,39.0,2024-10-05 00:00:00
720,org_001,user_000,data_export,deduct,90.0,2024-10-14 00:00:00
840,org_001,user_000,data_export,deduct,99.0,2024-11-01 00:00:00
660,org_001,user_000,api_call,deduct,38.0,2024-11-03 00:00:00
780,org_001,user_000,data_export,add,1.0,2024-11-10 12:00:00
...,...,...,...,...,...,...
639,org_004,user_019,data_export,deduct,63.0,1990-01-01 00:00:00
659,org_004,user_019,report_download,deduct,85.0,1990-01-01 00:00:00
679,org_004,user_019,api_call,deduct,38.0,1990-01-01 00:00:00
739,org_004,user_019,api_call,deduct,58.0,1990-01-01 00:00:00


In [19]:
# df['user_id'] = df['user_id'].fillna('user_999')
# df

In [20]:
# df = df.drop_duplicates(subset=['org_id', 'user_id', 'credit_type', 'timestamp'], keep='first')
# df

In [21]:
df = df.sort_values(['user_id', 'timestamp'])
first_orgs = df.groupby('user_id')['org_id'].transform('first')
df = df[df['org_id'] == first_orgs]
df

,org_id,user_id,credit_type,action,credits,timestamp
20,org_000,user_000,data_export,add,50.0,1990-01-01 00:00:00
80,org_000,user_000,report_download,deduct,95.0,1990-01-01 00:00:00
140,org_000,user_000,report_download,add,55.0,1990-01-01 00:00:00
260,org_000,user_000,report_download,add,45.0,1990-01-01 00:00:00
280,org_000,user_000,api_call,deduct,28.0,1990-01-01 00:00:00
...,...,...,...,...,...,...
799,org_004,user_019,api_call,add,66.0,2024-12-22 00:00:00
419,org_004,user_019,data_export,deduct,69.0,2024-12-23 00:00:00
519,org_004,user_019,api_call,deduct,86.0,2024-12-23 12:00:00
859,org_004,user_019,report_download,add,74.0,2024-12-27 12:00:00


In [25]:
df.dtypes

org_id                 object
user_id                object
credit_type            object
action                 object
credits               float64
timestamp      datetime64[ns]
dtype: object